In [5]:
import os
from glob import glob
import numpy as np
import pandas as pd

In [ ]:
raw_samples = glob('data/llah_raw_samples/*.txt')
col_names = ['ts', 'x', 'y', 'event', 'x_fix', 'y_fix', 'duration', 'word', 'doc']

In [ ]:
def convert_timestamp(ts):
    try:
        m, s, fs = [int(t) for t in ts.split(':')]
        t = (m*60) + (s*24) + fs 
        return t
    except:
        return None
    
def get_fixation(df):
    tf = pd.DataFrame(columns=df.columns)
    ix = 0
    for i, item in df.iterrows():
        if i > 0 and item.event != 'Fixation' and df.loc[i-1, 'event'] == 'Fixation':
            tf.loc[ix] = df.loc[i-1]
            ix += 1
    return tf

In [ ]:
data_frames = []
for raw in raw_samples:
    df = pd.read_csv(raw, sep='\t', names=col_names, header=0)
    df = df.drop('word', axis=1)
    df['participant'], df['condition'] = raw.split('P')[-1].split('.')[0].split('-')
    df.ts = df.ts.apply(convert_timestamp)
    df.doc = df.doc.fillna(method='ffilL')
    data_frames.append(df)
df = pd.concat(data_frames).reset_index(drop=True)


In [ ]:
df = get_fixation(df)

In [ ]:
df.to_csv('data/all_fix.csv', index=None)